In [8]:
#Input Variable
searchwords = ['kimchi','korean']
#Try these search words: ['korean', 'kimchi'], ['sushi', 'japan', 'sashimi', 'roll', 'udon'], ['spicy']
#If we can't assign searchwords to any restaurant cluster, we print 'Sorry, we did not understand your query!'. (i.e. ['peruvian'])

In [10]:
import nltk
nltk.download('wordnet')
import pandas as pd
import sqlite3
import operator 
import ast
from nltk.corpus import wordnet

df_preclustered = pd.read_csv('preclustered.csv')
df_term_cluster = pd.read_csv('restaurant_cluster_terms.csv')

THRESHOLD = 0.9
TOP_N = 20

cluster_terms = []
for cluster in df_term_cluster['terms']:
    t = ast.literal_eval(cluster)
    cluster_terms.append(t)
# i.e. cluster_terms = [['dog', 'dogs', 'fries', 'chili', 'chicago', 'italian', 'bun', 'beer', 'patio', 'onions'],
#          ['sushi', 'roll', 'ayce', 'rolls', 'nigiri', 'tuna', 'fish', 'rice', 'sashimi', 'tempura'],
#          ['burger', 'fries', 'burgers', 'bacon', 'shake', 'bun', 'patty', 'fast', 'chili', 'double'],
#          ['pizza', 'crust', 'pizzas', 'pepperoni', 'toppings', 'slice', 'garlic', 'delivery', 'pie', 'wings'],
#          ['steak', 'lobster', 'crab', 'filet', 'bread', 'shrimp', 'steakhouse', 'wine', 'bar', 'dessert'],
#          ['thai', 'pad', 'curry', 'rice', 'spicy', 'noodles', 'soup', 'tom', 'dish', 'beef'],
#          ['rice', 'filipino', 'ramen', 'poke', 'hawaiian', 'curry', 'pork', 'bowl', 'teriyaki', 'beef'],
#          ['sandwich', 'coffee', 'sandwiches', 'chocolate', 'bread', 'deli', 'cafe', 'soup', 'boba', 'pastries'],
#          ['chinese', 'rice', 'noodles', 'sum', 'soup', 'dim', 'noodle', 'pork', 'beef', 'shrimp'],
#          ['tacos', 'mexican', 'salsa', 'taco', 'burrito', 'chips', 'asada', 'carne', 'beans', 'guacamole'],
#          ['italian', 'pasta', 'bread', 'wine', 'dish', 'veal', 'meatballs', 'olive', 'lasagna', 'ravioli'],
#          ['pho', 'vietnamese', 'broth', 'rolls', 'pork', 'noodles', 'spring', 'rice', 'beef', 'egg'],
#          ['waffles', 'waffle', 'hash', 'breakfast', 'soul', 'eggs', 'gravy', 'mac', 'bacon', 'coffee'],
#          ['pita', 'hummus', 'greek', 'mediterranean', 'lamb', 'rice', 'fries', 'bread', 'beef', 'wrap'],
#          ['breakfast', 'eggs', 'pancakes', 'toast', 'coffee', 'hash', 'egg', 'omelet', 'steak', 'cafe'],
#          ['smoothie', 'healthy', 'juice', 'juices', 'wrap', 'vegan', 'protein', 'fries', 'bowl', 'sandwich'],
#          ['korean', 'bbq', 'pork', 'ayce', 'meats', 'beef', 'brisket', 'tofu', 'rice', 'grill'],
#          ['bar', 'beer', 'fries', 'bbq', 'burger', 'casino', 'ribs', 'hour', 'server', 'brisket'],
#          ['buffet', 'buffets', 'station', 'legs', 'crab', 'curry', 'sushi', 'selection', '99', 'prime'],
#          ['wings', 'wing', 'boneless', 'fries', 'buffalo', 'bar', 'garlic', 'sauces', 'sports', 'wild']]
    
result = []
searchwords_list = []
cluster_terms_list = []

for word1 in searchwords:
    if len(wordnet.synsets(word1)) != 0:
        searchwords_list.append(word1)

for i in range(len(cluster_terms)):
    temp = []
    for word2 in cluster_terms[i]:
        if len(wordnet.synsets(word2)) != 0:
            temp.append(word2)
    cluster_terms_list.append(temp)
    
clust = []
average = []
for i in range(len(cluster_terms_list)):
    #temp = []
    total = 0
    count = 0
    result = []
    for word2 in cluster_terms_list[i]:
        wordFromList2 = wordnet.synsets(word2)[0]
        for word1 in searchwords_list:
            wordFromList1 = wordnet.synsets(word1)[0]
            s = wordFromList2.wup_similarity(wordFromList1)
            if s!=None:
                if s > THRESHOLD:
                    result.append([word1, word2, s])
                    total += s
                    count += 1
    clust.append(result)
    if count>0:
        average.append(total/count)
    else:
        average.append([])
        
        
indices = []
similarity = []

for i in range(len(average)):
    if type(average[i]) is float:
        indices.append(i)
        similarity.append(average[i])   
rank = {'similarity': similarity, 'indices': indices}
df_rank = pd.DataFrame(rank, columns = {'similarity', 'indices'})

df_rank = df_rank.sort_values(by=['similarity'], ascending=False)
ranked_similarity = df_rank['similarity'].tolist()
ranked_indices = df_rank['indices'].tolist()
same_similarity_value = True

best_cluster = []
if len(ranked_similarity)==0:
    print('Sorry, we did not understand your query!')
else:
    best_score = ranked_similarity[0]
    for i in range(len(ranked_similarity)):
        if ranked_similarity[i] == best_score:
            best_cluster.append(ranked_indices[i])
        else:
            break
            
            
    print('Your Query: ', searchwords)
    for cluster in best_cluster:
        print('Best Cluster (based on your query): ', cluster)
        print('Topics: ', cluster_terms[cluster])

    conn = sqlite3.connect("yelp2.db")
    business = pd.read_sql_query("select * from business;", conn)

    result = []
    sorted_result = []
    for cluster in best_cluster:
        restaurants_stars = []
        restaruants_bestcluster = df_preclustered[df_preclustered['restaurant_cluster']==cluster]['business_id'].tolist()
        for el in set(restaruants_bestcluster):
            restaurants_stars.append([el, business[business['business_id']==el]['stars'].iloc[0]])
        sorted_result.append(sorted(restaurants_stars, key = lambda x: x[1]))

    final_restaurantids = []
    for cluster in sorted_result:
        if len(cluster)<TOP_N:
            TOP_N = len(cluster)
        for i in range(1, TOP_N+1):
            final_restaurantids.append(cluster[-i][0])

    final_list = []
    for restaurant in final_restaurantids:
        final_list.append([business[business['business_id'] == restaurant]['name'].iloc[0], 
                       business[business['business_id'] == restaurant]['categories'].iloc[0], 
                       business[business['business_id'] == restaurant]['address'].iloc[0]])

    print('\nRecommendations (ranked):')
    for i in range(len(final_list)):
        print(final_list[i])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/DaeHyunKim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Your Query:  ['kimchi', 'korean']
Best Cluster (based on your query):  16
Topics:  ['korean', 'bbq', 'pork', 'ayce', 'meats', 'beef', 'brisket', 'tofu', 'rice', 'grill']

Recommendations (ranked):
['Q Bistro', "['Barbeque', 'Restaurants', 'Nightlife', 'Bars', 'Asian Fusion', 'Chicken Wings', 'Korean']", '3400 S Jones Blvd, Ste 12']
['MAMAOH', "['Korean', 'Asian Fusion', 'Restaurants']", '8635 S Rainbow Blvd, Ste 110']
['Goong Korean BBQ Restaurant', "['Korean', 'Barbeque', 'Restaurants']", '7729 S Rainbow Blvd']
['Gangnam Asian BBQ Dining', "['Korean', 'Japanese', 'Restaurants', 'Tapas/Small Plates', 'Asian Fusion', 'Barbeque']", '4480 Paradise Rd']
['Magal BBQ', "['Barbeque', 'Korean', 'Restaurants']", '4240 Spring Mountain Rd']
['Hwaro', "['Korean', 'Basque', 'Restaurants', 'Barbeque']", '5030 Spring Mountain Rd']
['Island Style', "['Restauran